Okay! Let's try the gym business. Cliff walking environment is perfect.

In [1]:
import gym
import numpy as np
import ToyEnv

In [2]:
env = gym.make('CliffWalking-v0')

In [3]:
observation, info = env.reset()
print(observation)
print(info)
np.unravel_index(observation, env.shape)

36
{'prob': 1}


(3, 0)

In [4]:
action = env.action_space.sample()
print(action)
observation, reward, terminated, truncated, info = env.step(3)
print(observation)
print(env.action_space.n)
print(reward)
print(env.P[observation][2])

2
36
4
-1
[(1.0, 36, -1, False)]


/home/srayan/miniconda3/envs/tsmdp/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


# Testing ToyEnv

In [5]:
new_env = ToyEnv.ToyEnv()

In [7]:
init_state = new_env.reset()
print(init_state)

1


In [16]:
next_state, reward = new_env.step(1)

In [17]:
print(reward)
print(next_state)

3.081739534562946
2
